# **ETL process for Raw data**

## Objectives

* Extract dataset from Kaggle and import into notebook
* Push data through ETL pipeline to clean data using pandas and numpy
* Use Clean data to create visualisations using a variety of visualisation libraries
* Use feature engine to create predictive data and store in a predictive data file
* Use predictive date to create visualisations using visualisation libraries
* Create conclusions based off of visualisations and explain 

## Inputs

* Write down which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\naqas\\OneDrive\\Documents\\Coding\\CI_Projects\\london-housing-analysis-project\\london_housing_analysis_project\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\naqas\\OneDrive\\Documents\\Coding\\CI_Projects\\london-housing-analysis-project\\london_housing_analysis_project'

# Import required packages for notebook

Importing packages for cleaning

In [4]:
import pandas as pd
import numpy as np


---

# Import and clean dataset

Importing a data set with monthly values for house prices, houses sold and crimes committed for varying boroughs of London, regions of England and England in general. This is organised from 01/01/1995 to 01/01/2020.

In [10]:
monthly_variables_df = pd.read_csv("dataFiles/RawData/housing_in_london_monthly_variables.csv")
monthly_variables_df

,date,area,average_price,code,houses_sold,no_of_crimes,borough_flag
0,01/01/1995,city of london,91449,E09000001,17.0,NaN,1
1,01/02/1995,city of london,82203,E09000001,7.0,NaN,1
2,01/03/1995,city of london,79121,E09000001,14.0,NaN,1
3,01/04/1995,city of london,77101,E09000001,7.0,NaN,1
4,01/05/1995,city of london,84409,E09000001,10.0,NaN,1
...,...,...,...,...,...,...,...
13544,01/09/2019,england,249942,E92000001,64605.0,NaN,0
13545,01/10/2019,england,249376,E92000001,68677.0,NaN,0
13546,01/11/2019,england,248515,E92000001,67814.0,NaN,0
13547,01/12/2019,england,250410,E92000001,NaN,NaN,0


In [11]:
monthly_variables_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13549 entries, 0 to 13548
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           13549 non-null  object 
 1   area           13549 non-null  object 
 2   average_price  13549 non-null  int64  
 3   code           13549 non-null  object 
 4   houses_sold    13455 non-null  float64
 5   no_of_crimes   7439 non-null   float64
 6   borough_flag   13549 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 741.1+ KB


In [12]:
monthly_variables_df.isnull().sum()

date                0
area                0
average_price       0
code                0
houses_sold        94
no_of_crimes     6110
borough_flag        0
dtype: int64

### Removing empty/null values

The dataset contained a lot of null values for crime statistics in varying boroughs. By doing a quick analysis of the original csv file, I was able to determine that the null values only occurred prior to January 1st 2001. Also, due to the initial analysis of the data using .info() I was able to determine that the data had a range index. I then decided I should change the index to date-time and then clear the null values for all data prior to 01/01/01.

In [13]:
monthly_variables_df["date"] = pd.to_datetime(monthly_variables_df["date"])
monthly_variables_df = monthly_variables_df.set_index("date")
monthly_variables_df = monthly_variables_df.drop(monthly_variables_df[monthly_variables_df.index < "2001-01-01"].index)
monthly_variables_df

,area,average_price,code,houses_sold,no_of_crimes,borough_flag
date,,,,,,
2001-01-01,city of london,284262,E09000001,24.0,0.0,1
2001-01-02,city of london,198137,E09000001,37.0,0.0,1
2001-01-03,city of london,189033,E09000001,44.0,0.0,1
2001-01-04,city of london,205494,E09000001,38.0,0.0,1
2001-01-05,city of london,223459,E09000001,30.0,0.0,1
...,...,...,...,...,...,...
2019-01-09,england,249942,E92000001,64605.0,NaN,0
2019-01-10,england,249376,E92000001,68677.0,NaN,0
2019-01-11,england,248515,E92000001,67814.0,NaN,0


Now I will assess how many null values are left. This will not include any cells which contain "0".

In [14]:
monthly_variables_df.isnull().sum()

area                0
average_price       0
code                0
houses_sold        90
no_of_crimes     2866
borough_flag        0
dtype: int64

After assessing the new null values we can see that "no_of_crimes" fell from 6110 to 2866 and "houses_sold" fell from 94 to 90. There are still some null values and so further cleaning is required. Again, after assessing the original csv file, it is evident that there is a "borough flag" to help identify which entries are borough specific and which entries are not London boroughs. The non-borough rows also do not contain "Number of crime" statistics". By removing this information we can focus directly on the boroughs of London. The rows have to be removed via use of the "borough flag". "1" equals "True" and "0" equals "False" for identifying an entry as a borough.


In [15]:
monthly_variables_df = monthly_variables_df[monthly_variables_df["borough_flag"] != 0]
monthly_variables_df

,area,average_price,code,houses_sold,no_of_crimes,borough_flag
date,,,,,,
2001-01-01,city of london,284262,E09000001,24.0,0.0,1
2001-01-02,city of london,198137,E09000001,37.0,0.0,1
2001-01-03,city of london,189033,E09000001,44.0,0.0,1
2001-01-04,city of london,205494,E09000001,38.0,0.0,1
2001-01-05,city of london,223459,E09000001,30.0,0.0,1
...,...,...,...,...,...,...
2019-01-09,westminster,925955,E09000033,169.0,6384.0,1
2019-01-10,westminster,927864,E09000033,168.0,7208.0,1
2019-01-11,westminster,955615,E09000033,150.0,6843.0,1


In [16]:
monthly_variables_df.isnull().sum()

area               0
average_price      0
code               0
houses_sold       66
no_of_crimes     118
borough_flag       0
dtype: int64

Now, we can see that only 118 null values remain for "number of crimes" and 66 null values remain for "Houses sold". The remaining null entries for "number of crimes" are all entries related to the City of London. According to London's City council website, this is not a borough of London. This has been incorrectly marked as a borough in the dataset. This will need removing and then null value analysis done to see what null values remain.

In [17]:
monthly_variables_df = monthly_variables_df[monthly_variables_df["area"] != "city of london"]
monthly_variables_df

,area,average_price,code,houses_sold,no_of_crimes,borough_flag
date,,,,,,
2001-01-01,barking and dagenham,82343,E09000002,215.0,1771.0,1
2001-01-02,barking and dagenham,83266,E09000002,238.0,1600.0,1
2001-01-03,barking and dagenham,84058,E09000002,274.0,1759.0,1
2001-01-04,barking and dagenham,83965,E09000002,241.0,1643.0,1
2001-01-05,barking and dagenham,85124,E09000002,283.0,1809.0,1
...,...,...,...,...,...,...
2019-01-09,westminster,925955,E09000033,169.0,6384.0,1
2019-01-10,westminster,927864,E09000033,168.0,7208.0,1
2019-01-11,westminster,955615,E09000033,150.0,6843.0,1


In [18]:
monthly_variables_df.isnull().sum()

area              0
average_price     0
code              0
houses_sold      64
no_of_crimes      0
borough_flag      0
dtype: int64

Now the only null values remain within "houses sold". After investigating the original CSV, this was revealed to be the final two months of each data collection for each borough (December 2019 and January 2020). To prevent the data showing inaccuracies, this data will be removed.


In [19]:
monthly_variables_df = monthly_variables_df.dropna()

In [20]:
monthly_variables_df.isnull().sum()

area             0
average_price    0
code             0
houses_sold      0
no_of_crimes     0
borough_flag     0
dtype: int64

To aid with the data visualisation, a "year" column will be created as part of the data. This will help condense the data as an average per year and so help with drawing conclusions from the data.

In [21]:
monthly_variables_df["year"] = monthly_variables_df.index.year
monthly_variables_df

C:\Users\naqas\AppData\Local\Temp\ipykernel_12212\1676515305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_variables_df["year"] = monthly_variables_df.index.year


,area,average_price,code,houses_sold,no_of_crimes,borough_flag,year
date,,,,,,,
2001-01-01,barking and dagenham,82343,E09000002,215.0,1771.0,1,2001
2001-01-02,barking and dagenham,83266,E09000002,238.0,1600.0,1,2001
2001-01-03,barking and dagenham,84058,E09000002,274.0,1759.0,1,2001
2001-01-04,barking and dagenham,83965,E09000002,241.0,1643.0,1,2001
2001-01-05,barking and dagenham,85124,E09000002,283.0,1809.0,1,2001
...,...,...,...,...,...,...,...
2019-01-07,westminster,957897,E09000033,168.0,6991.0,1,2019
2019-01-08,westminster,914870,E09000033,156.0,7069.0,1,2019
2019-01-09,westminster,925955,E09000033,169.0,6384.0,1,2019


---

# Next Steps


* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.